## Guilty inventory report


In [1]:
import logging
import sqlite3

import pandas as pd

logging.basicConfig(
    level=logging.DEBUG, format=" %(asctime)s -  %(levelname)s -  %(message)s"
)

In [2]:
# def main():
#     create_tables()
#     t_sales_update("sales201908.csv")
#     t_sales_update("sales-201909-202008.csv")
#     t_inv_update("inv201909-202008.csv")
#     t_purchase_update("purchase201908-202008.csv")
#     t_business_unit_update("business_unit.csv")
#     t_rate_update("rate.csv")
#     t_aging_update("aging.csv")
#     t_guilty_base_generate()
#     t_guilty_base_update()
#     t_digest_update()
#     t_guilty_list_generate()
#     t_guilty_digest_generate()


In [3]:
def create_tables():
    # Connect to sqlite3
    try:
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Create tables
        # t_sales table
        c.execute(
            """
        create table if not exists t_sales (sales_month text, sales_model text,
        sales_qty int, sales_amt float, primary key(sales_month, sales_model))
        """
        )

        # t_purchase table
        c.execute(
            """
        create table if not exists t_purchase (purchase_month text, purchase_model text,
        purchase_qty int, purchase_amt float, primary key(purchase_month, purchase_model))
        """
        )

        # t_inv table
        c.execute(
            """
        create table if not exists t_inv (inv_month text, inv_model text, inv_cat int,
        inv_qty int, inv_amt float, inv_age int, primary key(inv_month, inv_model, inv_cat, inv_age))
        """
        )

        # t_business_unit table
        c.execute(
            """
        create table if not exists t_business_unit (prod_category int primary key,
        business_unit text, bu_description text)
        """
        )

        # t_rate table
        c.execute(
            """
        create table if not exists t_rate (rate_month text primary key,
        rate_yen float)
        """
        )

        # t_aging table
        c.execute(
            """
        create table if not exists t_aging (
        age_actual int primary key, age_n0 int, age_n1 int, age_n2 int, age_n3 int, age_n4 int,
        age_n5 int, age_n6 int, age_n7 int, age_n8 int, age_n9 int, age_n10 int, age_n11 int,
        age_n12 int
        )
        """
        )

        # t_digest_actage
        c.execute(
            """
        create table if not exists t_digest_actage (
        inv_month text, inv_model text, guilty_MYEN float, inv_model_digest text,
        business_unit text, primary key(inv_month, inv_model, business_unit))
        """
        )

        # t_digest_newage
        c.execute(
            """
        create table if not exists t_digest_newage (
        inv_month text, inv_model text, guilty_MYEN float, inv_model_digest text,
        business_unit text, primary key(inv_month, inv_model, business_unit))
        """
        )

        # t_guilty_base
        c.execute(
            """
        create table if not exists t_guilty_base (
        inv_month text, inv_model text, inv_cat int, business_unit text,
        inv_age int, inv_age_mod int, rate_yen float, inv_qty int, inv_KMXN float,
        inv_MYEN float, sales_n0_Q int, sales_n0_KMXN float, sales_n0_MYEN float, 
        sales_n1_Q int, sales_n1_KMXN float, sales_n1_MYEN float, 
        sales_n2_Q int, sales_n2_KMXN float, sales_n2_MYEN float,
        sales_n3_Q int, sales_n3_KMXN float, sales_n3_MYEN float, 
        sales_n4_Q int, sales_n4_KMXN float, sales_n4_MYEN float,
        sales_n5_Q int, sales_n5_KMXN float, sales_n5_MYEN float,
        sales_n6_Q int, sales_n6_KMXN float, sales_n6_MYEN float,
        sales_n7_Q int, sales_n7_KMXN float, sales_n7_MYEN float,
        sales_n8_Q int, sales_n8_KMXN float, sales_n8_MYEN float,
        sales_n9_Q int, sales_n9_KMXN float, sales_n9_MYEN float,
        sales_n10_Q int, sales_n10_KMXN float, sales_n10_MYEN float,
        sales_n11_Q int, sales_n11_KMXN float, sales_n11_MYEN float,
        sales_n12_Q int, sales_n12_KMXN float, sales_n12_MYEN float,
        purchase_n0_Q int, purchase_n1_Q int, purchase_n2_Q int, purchase_n3_Q int,
        purchase_n4_Q int, purchase_n5_Q int, purchase_n6_Q int, purchase_n7_Q int, 
        purchase_n8_Q int, purchase_n9_Q int, purchase_n10_Q int, purchase_n11_Q int, 
        purchase_n12_Q int, primary key(inv_month, inv_model, inv_age))
        """
        )

        # t_guilty_list_actage
        c.execute(
            """
        create table if not exists t_guilty_list_actage (
        inv_month text, inv_model text, business_unit text,
        inv_age int, inv_qty int, inv_KMXN float, inv_MYEN float,
        primary key(inv_month, inv_model, inv_age))
        """
        )

        # t_guilty_list_newage
        c.execute(
            """
        create table if not exists t_guilty_list_newage (
        inv_month text, inv_model text, business_unit text,
        inv_age_mod int, inv_qty int, inv_KMXN float, inv_MYEN float,
        primary key(inv_month, inv_model, inv_age_mod))
        """
        )

        # t_guilty_inv_sum_actage
        c.execute(
            """
        create table if not exists t_guilty_inv_sum_actage (
        g_month text, g_model text, g_business_unit, g_qty int, primary key(g_month, g_model, g_business_unit))

        """
        )

        # t_guilty_inv_sum_newage
        c.execute(
            """
        create table if not exists t_guilty_inv_sum_newage (
        g_month text, g_model text, g_business_unit, g_qty int, primary key(g_month, g_model, g_business_unit))

        """
        )

        # t_guilty_digest_actage
        c.execute(
            """
        create table if not exists t_guilty_digest_actage (
        inv_month text, inv_model_digest text, business_unit text, inv_qty int,
        inv_KMXN float, inv_n1_Q int, sales_n0_Q int, sales_n0_KMXN float,
        sales_n1_Q int, sales_n1_KMXN float, sales_n2_Q int, sales_n2_KMXN float,
        sales_n3_Q int, sales_n3_KMXN float, sales_n4_Q int, sales_n4_KMXN float,
        sales_n5_Q int, sales_n5_KMXN float,sales_n6_Q int, sales_n6_KMXN float,
        sales_n7_Q int, sales_n7_KMXN float,sales_n8_Q int, sales_n8_KMXN float,
        sales_n9_Q int, sales_n9_KMXN float,sales_n10_Q int, sales_n10_KMXN float,
        sales_n11_Q int, sales_n11_KMXN float,sales_n12_Q int, sales_n12_KMXN float,
        primary key(inv_month, inv_model_digest, business_unit))
        """
        )

        # t_guilty_digest_newage
        c.execute(
            """
        create table if not exists t_guilty_digest_newage (
        inv_month text, inv_model_digest text, business_unit text, inv_qty int,
        inv_KMXN float, inv_n1_Q int, sales_n0_Q int, sales_n0_KMXN float,
        sales_n1_Q int, sales_n1_KMXN float, sales_n2_Q int, sales_n2_KMXN float,
        sales_n3_Q int, sales_n3_KMXN float, sales_n4_Q int, sales_n4_KMXN float,
        sales_n5_Q int, sales_n5_KMXN float,sales_n6_Q int, sales_n6_KMXN float,
        sales_n7_Q int, sales_n7_KMXN float,sales_n8_Q int, sales_n8_KMXN float,
        sales_n9_Q int, sales_n9_KMXN float,sales_n10_Q int, sales_n10_KMXN float,
        sales_n11_Q int, sales_n11_KMXN float,sales_n12_Q int, sales_n12_KMXN float,
        primary key(inv_month, inv_model_digest, business_unit))
        """
        )

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


create_tables()

Successfully finished without error


In [4]:
def t_sales_update(targetFile):
    # Sales record preparation from csv and add to sqlite3 t_sales table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Billing doc. date must be formatted to date (ex: 2020/8/2)
    # Sort by Billing doc. is ideal.

    useCols = ["Billing doc. date", "Material", "Net Qty", "Net $"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[0])
    df = df.rename(
        columns={
            "Billing doc. date": "sales_month",
            "Material": "sales_model",
            "Net Qty": "sales_qty",
            "Net $": "sales_amt",
        }
    )
    
    # Modify the sales month to have 1st date of the month
    df["sales_month"] = df["sales_month"].apply(lambda x: x.replace(day=1))
    grouped = df.groupby(["sales_month", "sales_model"], as_index=False)
    # print(grouped.sum())

    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_sales table
        grouped.sum().to_sql("t_sales", conn, if_exists="append", index=None)
    except Exception as e:
        print(e)
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_sales_update("sales201908.csv")
t_sales_update("sales-201909-202008.csv")

Done with no error
Done with no error


In [5]:
def t_inv_update(targetFile):
    # Inventory aging preparation from csv and add to sqlite3 t_inv table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Add columun "inv_month" with date format like 2019/9/1 (day must be 1st day)

    useCols = ["CatNo", "Material", "Fecha", "Suma de Total Qty", "Suma de Total $", "inv_month"]

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[5])
    df = df.rename(
        columns={
            "CatNo": "inv_cat",
            "Material": "inv_model",
            "Fecha": "inv_age",
            "Suma de Total Qty": "inv_qty",
            "Suma de Total $": "inv_amt"
        }
    )
    
    
    df = df.reindex(columns=['inv_month', 'inv_model', 'inv_cat', 'inv_qty', 'inv_amt', 'inv_age'])
    
    # Replace the aging date
    agedict = {'30D': 30, '60D': 60, '90D': 90, '180D': 180, '270D': 270, '360D': 360, 'MAS': 999}
    df = df.replace(agedict)
  
    # Modify the inv_month to have 1st date of the month, just in case.
    df["inv_month"] = df["inv_month"].apply(lambda x: x.replace(day=1))
    
    # logging.debug('df status :\n' + str(df))
    # logging.debug('df is' + str(df['inv_month'].dtype))

    # Grouping by in_month and inv_model
    grouped = df.groupby(["inv_month", "inv_model", "inv_cat", "inv_age"], as_index=False)
    # logging.debug('grouped.sum() status:\n' + str(grouped.sum()))

     # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_inv table
        grouped.sum().to_sql("t_inv", conn, if_exists="append", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_inv_update("inv201909-202008.csv")

Done with no error


In [6]:
def t_purchase_update(targetFile):
    # Purchase preparation from csv and add to sqlite3 t_purchase table
    # Copy the original sheet to a new sheet & paste value and save as csv.

    useCols = ["Material", "Posting date", "Net Qty", "Net - $"]

    df = pd.read_csv(targetFile, header=0, usecols=useCols, parse_dates=[1])
    df = df.rename(
        columns={
            "Material": "purchase_model",
            "Net Qty": "purchase_qty",
            "Net - $": "purchase_amt",
            "Posting date": "purchase_month"
        }
    )
    
    
    df = df.reindex(columns=['purchase_month', 'purchase_model', 'purchase_qty', 'purchase_amt'])
    
    # Modify the purchase_month to have 1st date of the month
    df["purchase_month"] = df["purchase_month"].apply(lambda x: x.replace(day=1))
    
#     logging.debug('df status :\n' + str(df))
#     logging.debug('df is' + str(df['purchase_month'].dtype))

    # Grouping by in_month and inv_model
    grouped = df.groupby(["purchase_month", "purchase_model"], as_index=False)
#    logging.debug('grouped.sum() status:\n' + str(grouped.sum()))

     # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add grouped.sum() into t_purchase table
        grouped.sum().to_sql("t_purchase", conn, if_exists="append", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_purchase_update("purchase201908-202008.csv")

Done with no error


In [7]:
def t_business_unit_update(targetFile):
    # business_unit preparation from csv and add to sqlite3 t_business_unit table

    df = pd.read_csv(targetFile)
    
    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add business unit into t_business_unit table
        df.to_sql("t_business_unit", conn, if_exists="replace", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_business_unit_update("business_unit.csv")

Done with no error


In [8]:
def t_rate_update(targetFile):
    # Rate table preparation from csv and add to sqlite3 t_rate table

    df = pd.read_csv(targetFile, parse_dates=[0])
        
    # Modify the inv_month to have 1st date of the month, just in case.
    df["rate_month"] = df["rate_month"].apply(lambda x: x.replace(day=1))

    
    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add rate into t_rate table
        df.to_sql("t_rate", conn, if_exists="replace", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_rate_update("rate.csv")

Done with no error


In [9]:
def t_aging_update(targetFile):
    # Rate table preparation from csv and add to sqlite3 t_aging table

    df = pd.read_csv(targetFile)
        
    # Connect to sqlite3
    conn = sqlite3.connect("guilty_inventory.sqlite3")
    c = conn.cursor()

    try:
        # Add aging into t_aging table
        df.to_sql("t_aging", conn, if_exists="replace", index=None)
    except Exception as e:
        print('Error : ' + str(e))
    else:
        print("Done with no error")

    # Close connection to sqlite3
    c.close()
    conn.close()


t_aging_update("aging.csv")

Done with no error


In [10]:
def t_guilty_base_generate():

    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Clear data in t_guilty_base
        c.execute(
            """
        delete from t_guilty_base
        """
        )
        # Add data into t_guilty_base
        c.execute(
            """
        insert into t_guilty_base
        SELECT 
        inv_month,
        inv_model,
        inv_cat,
        business_unit,
        inv_age,
        inv_age as inv_age_mod,
        rate_yen,
        inv_qty,
        inv_amt / 1000 as inv_KMXN,
        inv_amt * rate_yen / 1000000 as inv_MYEN,

        sn0.sales_qty as sales_n0_Q,
        sn0.sales_amt / 1000 as sales_n0_KMXN,
        sn0.sales_amt * rate_yen / 1000000 as sales_n0_MYEN,

        sn1.sales_qty as sales_n1_Q,
        sn1.sales_amt / 1000 as sales_n1_KMXN,
        sn1.sales_amt * rate_yen / 1000000 as sales_n1_MYEN,

        sn2.sales_qty as sales_n2_Q,
        sn2.sales_amt / 1000 as sales_n2_KMXN,
        sn2.sales_amt * rate_yen / 1000000 as sales_n2_MYEN,

        sn3.sales_qty as sales_n3_Q,
        sn3.sales_amt / 1000 as sales_n3_KMXN,
        sn3.sales_amt * rate_yen / 1000000 as sales_n3_MYEN,

        sn4.sales_qty as sales_n4_Q,
        sn4.sales_amt / 1000 as sales_n4_KMXN,
        sn4.sales_amt * rate_yen / 1000000 as sales_n4_MYEN,

        sn5.sales_qty as sales_n5_Q,
        sn5.sales_amt / 1000 as sales_n5_KMXN,
        sn5.sales_amt * rate_yen / 1000000 as sales_n5_MYEN,

        sn6.sales_qty as sales_n6_Q,
        sn6.sales_amt / 1000 as sales_n6_KMXN,
        sn6.sales_amt * rate_yen / 1000000 as sales_n6_MYEN,

        sn7.sales_qty as sales_n7_Q,
        sn7.sales_amt / 1000 as sales_n7_KMXN,
        sn7.sales_amt * rate_yen / 1000000 as sales_n7_MYEN,

        sn8.sales_qty as sales_n8_Q,
        sn8.sales_amt / 1000 as sales_n8_KMXN,
        sn8.sales_amt * rate_yen / 1000000 as sales_n8_MYEN,

        sn9.sales_qty as sales_n9_Q,
        sn9.sales_amt / 1000 as sales_n9_KMXN,
        sn9.sales_amt * rate_yen / 1000000 as sales_n9_MYEN,

        sn10.sales_qty as sales_n10_Q,
        sn10.sales_amt / 1000 as sales_n10_KMXN,
        sn10.sales_amt * rate_yen / 1000000 as sales_n10_MYEN,

        sn11.sales_qty as sales_n11_Q,
        sn11.sales_amt / 1000 as sales_n11_KMXN,
        sn11.sales_amt * rate_yen / 1000000 as sales_n11_MYEN,

        sn12.sales_qty as sales_n12_Q,
        sn12.sales_amt / 1000 as sales_n12_KMXN,
        sn12.sales_amt * rate_yen / 1000000 as sales_n12_MYEN,


        pn0.purchase_qty as purchase_n0_Q,
        pn1.purchase_qty as purchase_n1_Q,
        pn2.purchase_qty as purchase_n2_Q,
        pn3.purchase_qty as purchase_n3_Q,
        pn4.purchase_qty as purchase_n4_Q,
        pn5.purchase_qty as purchase_n5_Q,
        pn6.purchase_qty as purchase_n6_Q,
        pn7.purchase_qty as purchase_n7_Q,
        pn8.purchase_qty as purchase_n8_Q,
        pn9.purchase_qty as purchase_n9_Q,
        pn10.purchase_qty as purchase_n10_Q,
        pn11.purchase_qty as purchase_n11_Q,
        pn12.purchase_qty as purchase_n12_Q

        FROM t_inv 

        left outer join
        t_business_unit
        on inv_cat = prod_category
        
        left outer join
        t_rate
        on inv_month = rate_month

        left outer join
        t_sales sn0
        on inv_month = sn0.sales_month and inv_model = sn0.sales_model

        left outer join
        t_sales sn1
        on datetime(inv_month, '-1 months') = sn1.sales_month and inv_model = sn1.sales_model

        left outer join
        t_sales sn2
        on datetime(inv_month, '-2 months') = sn2.sales_month and inv_model = sn2.sales_model

        left outer join
        t_sales sn3
        on datetime(inv_month, '-3 months') = sn3.sales_month and inv_model = sn3.sales_model

        left outer join
        t_sales sn4
        on datetime(inv_month, '-4 months') = sn4.sales_month and inv_model = sn4.sales_model

        left outer join
        t_sales sn5
        on datetime(inv_month, '-5 months') = sn5.sales_month and inv_model = sn5.sales_model

        left outer join
        t_sales sn6
        on datetime(inv_month, '-6 months') = sn6.sales_month and inv_model = sn6.sales_model

        left outer join
        t_sales sn7
        on datetime(inv_month, '-7 months') = sn7.sales_month and inv_model = sn7.sales_model

        left outer join
        t_sales sn8
        on datetime(inv_month, '-8 months') = sn8.sales_month and inv_model = sn8.sales_model

        left outer join
        t_sales sn9
        on datetime(inv_month, '-9 months') = sn9.sales_month and inv_model = sn9.sales_model

        left outer join
        t_sales sn10
        on datetime(inv_month, '-10 months') = sn10.sales_month and inv_model = sn10.sales_model

        left outer join
        t_sales sn11
        on datetime(inv_month, '-11 months') = sn11.sales_month and inv_model = sn11.sales_model

        left outer join
        t_sales sn12
        on datetime(inv_month, '-12 months') = sn12.sales_month and inv_model = sn12.sales_model

        left outer join
        t_purchase pn0
        on inv_month = pn0.purchase_month and inv_model = pn0.purchase_model

        left outer join
        t_purchase pn1
        on datetime(inv_month, '-1 months') = pn1.purchase_month and inv_model = pn1.purchase_model

        left outer join
        t_purchase pn2
        on datetime(inv_month, '-2 months') = pn2.purchase_month and inv_model = pn2.purchase_model

        left outer join
        t_purchase pn3
        on datetime(inv_month, '-3 months') = pn3.purchase_month and inv_model = pn3.purchase_model

        left outer join
        t_purchase pn4
        on datetime(inv_month, '-4 months') = pn4.purchase_month and inv_model = pn4.purchase_model

        left outer join
        t_purchase pn5
        on datetime(inv_month, '-5 months') = pn5.purchase_month and inv_model = pn5.purchase_model

        left outer join
        t_purchase pn6
        on datetime(inv_month, '-6 months') = pn6.purchase_month and inv_model = pn6.purchase_model

        left outer join
        t_purchase pn7
        on datetime(inv_month, '-7 months') = pn7.purchase_month and inv_model = pn7.purchase_model
        
        left outer join
        t_purchase pn8
        on datetime(inv_month, '-8 months') = pn8.purchase_month and inv_model = pn8.purchase_model

        left outer join
        t_purchase pn9
        on datetime(inv_month, '-9 months') = pn9.purchase_month and inv_model = pn9.purchase_model

        left outer join
        t_purchase pn10
        on datetime(inv_month, '-10 months') = pn10.purchase_month and inv_model = pn10.purchase_model

        left outer join
        t_purchase pn11
        on datetime(inv_month, '-11 months') = pn11.purchase_month and inv_model = pn11.purchase_model

        left outer join
        t_purchase pn12
        on datetime(inv_month, '-12 months') = pn12.purchase_month and inv_model = pn12.purchase_model


        """
        )

        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_guilty_base_generate()

Successfully finished without error


In [11]:
def t_guilty_base_update():
    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Update inv_age_mod of t_guilty_base according to latest purchase and sales.
        c.execute(
            """
            UPDATE
            t_guilty_base
                SET
                    inv_age_mod = (
                        SELECT
                            CASE
                                when sales_n0_Q > 0
                                or purchase_n0_Q > 0 then age_n0
                                when sales_n1_Q > 0
                                or purchase_n1_Q > 0 then age_n1
                                when sales_n2_Q > 0
                                or purchase_n2_Q > 0 then age_n2
                                when sales_n3_Q > 0
                                or purchase_n3_Q > 0 then age_n3
                                when sales_n4_Q > 0
                                or purchase_n4_Q > 0 then age_n4
                                when sales_n5_Q > 0
                                or purchase_n5_Q > 0 then age_n5
                                when sales_n6_Q > 0
                                or purchase_n6_Q > 0 then age_n6
                                when sales_n7_Q > 0
                                or purchase_n7_Q > 0 then age_n7
                                when sales_n8_Q > 0
                                or purchase_n8_Q > 0 then age_n8
                                when sales_n9_Q > 0
                                or purchase_n9_Q > 0 then age_n9
                                when sales_n10_Q > 0
                                or purchase_n10_Q > 0 then age_n10
                                when sales_n11_Q > 0
                                or purchase_n11_Q > 0 then age_n11
                                when sales_n12_Q > 0
                                or purchase_n12_Q > 0 then age_n12
                                ELSE inv_age
                            END
                        FROM
                            t_aging
                        WHERE
                            inv_age = age_actual
                    )
                WHERE
                    EXISTS(
                        SELECT
                            *
                        FROM
                            t_aging
                        WHERE
                            inv_age = age_actual
                    )
            """
        )

        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_guilty_base_update()

Successfully finished without error


In [12]:
def t_digest_update():
    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Update inv_model_model_digest to "Other" if inv_age > 90 and sum(inv_MYEN < 0)
        # group by  inv_model.
        
        # Clear data in t_other_actage and add data
        c.execute(
            """
        delete from t_digest_actage
        """
        )

        c.execute(
            """
            insert into t_digest_actage
            select inv_month, inv_model, sum(inv_MYEN) as guilty_MYEN,
            CASE
                when sum(inv_MYEN) < 1 then 'Other' else inv_model
            END as inv_model_digest, business_unit
            
            from t_guilty_base where inv_age > 90 group by inv_month, inv_model
            """
        )

       # Clear data in t_other_newage and add data
        c.execute(
            """
        delete from t_digest_newage
        """
        )

        c.execute(
            """
            insert into t_digest_newage
            select inv_month, inv_model, sum(inv_MYEN) as guilty_MYEN,
            CASE
                when sum(inv_MYEN) < 1 then 'Other' else inv_model
            END as inv_model_digest, business_unit

            from t_guilty_base where inv_age_mod > 90 group by inv_month, inv_model
            """
        )

        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_digest_update()

Successfully finished without error


In [13]:
def t_guilty_list_generate():

    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Clear data in t_guilty_list_actage
        c.execute(
            """
        delete from t_guilty_list_actage
        """
        )
        # Add data into t_guilty_list_actage
        c.execute(
            """
        insert into t_guilty_list_actage
        SELECT
            inv_month,
            inv_model,
            business_unit,
            inv_age,
            inv_qty,
            inv_KMXN,
            inv_MYEN
        FROM
            t_guilty_base
        """
        )

        # Clear data in t_guilty_list_newage
        c.execute(
            """
        delete from t_guilty_list_newage
        """
        )

        # Add data into t_guilty_list_newage
        c.execute(
            """
        insert into t_guilty_list_newage
        SELECT
            inv_month,
            inv_model,
            business_unit,
            inv_age_mod,
            sum(inv_qty) as inv_qty,
            sum(inv_KMXN) as inv_KMXN,
            sum(inv_MYEN) as inv_MYEN
        FROM
            t_guilty_base group by inv_month, inv_model, business_unit, inv_age_mod
        """
        )

        
        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_guilty_list_generate()

Successfully finished without error


In [14]:
def t_guilty_digest_generate():

    try:
        # Connect to sqlite3
        conn = sqlite3.connect("guilty_inventory.sqlite3")
        c = conn.cursor()

        # Clear data in t_guilty_digest_actage
        c.execute(
            """
        delete from t_guilty_digest_actage
        """
        )
        # Add data into t_guilty_digest_actage
        c.execute(
            """
        insert into t_guilty_digest_actage
        SELECT
            tda.inv_month,
            tda.inv_model_digest,
            tda.business_unit,
            sum(inv_qty) as inv_qty,
            sum(inv_KMXN) as inv_KMXN,
            sum(inv_qty) as inv_n1_Q,
            
            sum(sn0.sales_qty) as sales_n0_Q,
            sum(sn0.sales_amt) / 1000 as sales_n0_KMXN,

            sum(sn1.sales_qty) as sales_n1_Q,
            sum(sn1.sales_amt) / 1000 as sales_n1_KMXN,

            sum(sn2.sales_qty) as sales_n2_Q,
            sum(sn2.sales_amt) / 1000 as sales_n2_KMXN,

            sum(sn3.sales_qty) as sales_n3_Q,
            sum(sn3.sales_amt) / 1000 as sales_n3_KMXN,

            sum(sn4.sales_qty) as sales_n4_Q,
            sum(sn4.sales_amt) / 1000 as sales_n4_KMXN,

            sum(sn5.sales_qty) as sales_n5_Q,
            sum(sn5.sales_amt) / 1000 as sales_n5_KMXN,

            sum(sn6.sales_qty) as sales_n6_Q,
            sum(sn6.sales_amt) / 1000 as sales_n6_KMXN,

            sum(sn7.sales_qty) as sales_n7_Q,
            sum(sn7.sales_amt) / 1000 as sales_n7_KMXN,

            sum(sn8.sales_qty) as sales_n8_Q,
            sum(sn8.sales_amt) / 1000 as sales_n8_KMXN,

            sum(sn9.sales_qty) as sales_n9_Q,
            sum(sn9.sales_amt) / 1000 as sales_n9_KMXN,

            sum(sn10.sales_qty) as sales_n10_Q,
            sum(sn10.sales_amt) / 1000 as sales_n10_KMXN,

            sum(sn11.sales_qty) as sales_n11_Q,
            sum(sn11.sales_amt) / 1000 as sales_n11_KMXN,

            sum(sn12.sales_qty) as sales_n12_Q,
            sum(sn12.sales_amt) / 1000 as sales_n12_KMXN

        FROM
            t_digest_actage tda
            left outer join t_guilty_base tg on tda.inv_month = tg.inv_month
            and tda.inv_model = tg.inv_model
            left outer join t_sales sn0 on tda.inv_month = sn0.sales_month
            and tda.inv_model = sn0.sales_model
            left outer join t_sales sn1 on datetime(tda.inv_month, '-1 months') = sn1.sales_month
            and tda.inv_model = sn1.sales_model
            left outer join t_sales sn2 on datetime(tda.inv_month, '-2 months') = sn2.sales_month
            and tda.inv_model = sn2.sales_model
            left outer join t_sales sn3 on datetime(tda.inv_month, '-3 months') = sn3.sales_month
            and tda.inv_model = sn3.sales_model
            left outer join t_sales sn4 on datetime(tda.inv_month, '-4 months') = sn4.sales_month
            and tda.inv_model = sn4.sales_model
            left outer join t_sales sn5 on datetime(tda.inv_month, '-5 months') = sn5.sales_month
            and tda.inv_model = sn5.sales_model
            left outer join t_sales sn6 on datetime(tda.inv_month, '-6 months') = sn6.sales_month
            and tda.inv_model = sn6.sales_model
            left outer join t_sales sn7 on datetime(tda.inv_month, '-7 months') = sn7.sales_month
            and tda.inv_model = sn7.sales_model
            left outer join t_sales sn8 on datetime(tda.inv_month, '-8 months') = sn8.sales_month
            and tda.inv_model = sn8.sales_model
            left outer join t_sales sn9 on datetime(tda.inv_month, '-9 months') = sn9.sales_month
            and tda.inv_model = sn9.sales_model
            left outer join t_sales sn10 on datetime(tda.inv_month, '-10 months') = sn10.sales_month
            and tda.inv_model = sn10.sales_model
            left outer join t_sales sn11 on datetime(tda.inv_month, '-11 months') = sn11.sales_month
            and tda.inv_model = sn11.sales_model
            left outer join t_sales sn12 on datetime(tda.inv_month, '-12 months') = sn12.sales_month
            and tda.inv_model = sn12.sales_model
        where
            tg.inv_age > 90
        group by
            tda.inv_month,
            tda.inv_model_digest,
            tda.business_unit        """
        )
        
        # Clear data in t_guilty_inv_sum_actage
        c.execute(
            """
        delete from t_guilty_inv_sum_actage
        """
        )

        # update t_guilty_inv_sum_actage to update t_guilty_digest_actage inv_n1_Q
        c.execute(
            """
            insert into t_guilty_inv_sum_actage
            select inv_month, inv_model_digest, business_unit,inv_qty
            from t_guilty_digest_actage
        """
        )

        # update t_guilty_digest_actage inv_n1_Q
        c.execute(
            """
        update t_guilty_digest_actage
            set inv_n1_Q = (
                select g_qty from t_guilty_inv_sum_actage
                where datetime(inv_month, '-1 months') = g_month
                and inv_model_digest = g_model and business_unit = g_business_unit)
        where EXISTS(
                select * from t_guilty_inv_sum_actage
                where datetime(inv_month, '-1 months') = g_month
                and inv_model_digest = g_model and business_unit = g_business_unit)
        """
        )


        # Clear data in t_guilty_digest_newage
        c.execute(
            """
        delete from t_guilty_digest_newage
        """
        )

        # Add data into t_guilty_digest_newage
        c.execute(
            """
        insert into t_guilty_digest_newage
        SELECT
            tdn.inv_month,
            tdn.inv_model_digest,
            tdn.business_unit,
            sum(inv_qty) as inv_qty,
            sum(inv_KMXN) as inv_KMXN,
            sum(inv_qty) as inv_n1_Q,
            sum(sn0.sales_qty) as sales_n0_Q,
            sum(sn0.sales_amt) / 1000 as sales_n0_KMXN,
            sum(sn1.sales_qty) as sales_n1_Q,
            sum(sn1.sales_amt) / 1000 as sales_n1_KMXN,
            sum(sn2.sales_qty) as sales_n2_Q,
            sum(sn2.sales_amt) / 1000 as sales_n2_KMXN,
            sum(sn3.sales_qty) as sales_n3_Q,
            sum(sn3.sales_amt) / 1000 as sales_n3_KMXN,
            sum(sn4.sales_qty) as sales_n4_Q,
            sum(sn4.sales_amt) / 1000 as sales_n4_KMXN,
            sum(sn5.sales_qty) as sales_n5_Q,
            sum(sn5.sales_amt) / 1000 as sales_n5_KMXN,
            sum(sn6.sales_qty) as sales_n6_Q,
            sum(sn6.sales_amt) / 1000 as sales_n6_KMXN,
            sum(sn7.sales_qty) as sales_n7_Q,
            sum(sn7.sales_amt) / 1000 as sales_n7_KMXN,
            sum(sn8.sales_qty) as sales_n8_Q,
            sum(sn8.sales_amt) / 1000 as sales_n8_KMXN,
            sum(sn9.sales_qty) as sales_n9_Q,
            sum(sn9.sales_amt) / 1000 as sales_n9_KMXN,
            sum(sn10.sales_qty) as sales_n10_Q,
            sum(sn10.sales_amt) / 1000 as sales_n10_KMXN,
            sum(sn11.sales_qty) as sales_n11_Q,
            sum(sn11.sales_amt) / 1000 as sales_n11_KMXN,
            sum(sn12.sales_qty) as sales_n12_Q,
            sum(sn12.sales_amt) / 1000 as sales_n12_KMXN
        FROM
            t_digest_newage tdn
            left outer join t_guilty_base tg on tdn.inv_month = tg.inv_month
            and tdn.inv_model = tg.inv_model
            left outer join t_sales sn0 on tdn.inv_month = sn0.sales_month
            and tdn.inv_model = sn0.sales_model
            left outer join t_sales sn1 on datetime(tdn.inv_month, '-1 months') = sn1.sales_month
            and tdn.inv_model = sn1.sales_model
            left outer join t_sales sn2 on datetime(tdn.inv_month, '-2 months') = sn2.sales_month
            and tdn.inv_model = sn2.sales_model
            left outer join t_sales sn3 on datetime(tdn.inv_month, '-3 months') = sn3.sales_month
            and tdn.inv_model = sn3.sales_model
            left outer join t_sales sn4 on datetime(tdn.inv_month, '-4 months') = sn4.sales_month
            and tdn.inv_model = sn4.sales_model
            left outer join t_sales sn5 on datetime(tdn.inv_month, '-5 months') = sn5.sales_month
            and tdn.inv_model = sn5.sales_model
            left outer join t_sales sn6 on datetime(tdn.inv_month, '-6 months') = sn6.sales_month
            and tdn.inv_model = sn6.sales_model
            left outer join t_sales sn7 on datetime(tdn.inv_month, '-7 months') = sn7.sales_month
            and tdn.inv_model = sn7.sales_model
            left outer join t_sales sn8 on datetime(tdn.inv_month, '-8 months') = sn8.sales_month
            and tdn.inv_model = sn8.sales_model
            left outer join t_sales sn9 on datetime(tdn.inv_month, '-9 months') = sn9.sales_month
            and tdn.inv_model = sn9.sales_model
            left outer join t_sales sn10 on datetime(tdn.inv_month, '-10 months') = sn10.sales_month
            and tdn.inv_model = sn10.sales_model
            left outer join t_sales sn11 on datetime(tdn.inv_month, '-11 months') = sn11.sales_month
            and tdn.inv_model = sn11.sales_model
            left outer join t_sales sn12 on datetime(tdn.inv_month, '-12 months') = sn12.sales_month
            and tdn.inv_model = sn12.sales_model
        where
            tg.inv_age > 90
        group by
            tdn.inv_month,
            tdn.inv_model_digest,
            tdn.business_unit        """
        )
        
        # Clear data in t_guilty_inv_sum_newage
        c.execute(
            """
        delete from t_guilty_inv_sum_newage
        """
        )


        # update t_guilty_inv_sum_newage to update t_guilty_digest_newage inv_n1_Q
        c.execute(
            """
        insert into t_guilty_inv_sum_newage
        select inv_month, inv_model_digest, business_unit, inv_qty
        from t_guilty_digest_newage        """
        )

        # update t_guilty_digest_newage inv_n1_Q
        c.execute(
            """
        update t_guilty_digest_newage
            set inv_n1_Q = (
                select g_qty from t_guilty_inv_sum_newage
                where datetime(inv_month, '-1 months') = g_month
                and inv_model_digest = g_model and business_unit = g_business_unit)
        where EXISTS(
                select * from t_guilty_inv_sum_newage
                where datetime(inv_month, '-1 months') = g_month
                and inv_model_digest = g_model and business_unit = g_business_unit)

        """
        )
        
        
        # Commit
        conn.commit()

        # Close connection to sqlite3
        c.close()
        conn.close()

    except Exception as e:
        print(e)

    else:
        print("Successfully finished without error")


t_guilty_digest_generate()

Successfully finished without error


In [15]:
# main()